In [2]:
#@title Installs

!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text --quiet
!pip install transformers --quiet

In [3]:
#!pip install tensorflow_text

import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
#import tensorflow_text as tf_text
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, TFBertModel
from transformers import logging
logging.set_verbosity_error()

from sklearn.feature_extraction.text import TfidfVectorizer

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import re

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

import nltk
from nltk.data import find
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('omw-1.4')
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import ne_chunk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import string
from collections import Counter
stopwords = set(stopwords.words('english'))


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wo

In [20]:
import pandas as pd

file_paths = ['/content/AM.csv', '/content/CF.csv', '/content/DB.csv', '/content/IB.csv']

df = [pd.read_csv(file_path) for file_path in file_paths]

merged = pd.concat(df, ignore_index = True)

print(merged)

       Label                                      Risk Sentence
0          0  risk factor risk factor normal course business...
1          0  following discussion material risk factor appl...
2          0  additional information risk management framewo...
3          0  additional risk beyond described management di...
4          0  strategic risk consummation planned acquisitio...
...      ...                                                ...
25532      2  dilution reduction delay accretion schwabs ear...
25533      2  pending merger td ameritrade completed stockho...
25534      2  proposed merger completed issue td ameritrade ...
25535      2  result issuance share common stock stockholder...
25536      2        addition td bank become largest stockholder

[25537 rows x 2 columns]


In [21]:
merged['Label'] = merged['Label'].astype(int)
label_counts = merged['Label'].value_counts()
print(label_counts)

2    7020
3    6610
0    6325
1    5582
Name: Label, dtype: int64


In [22]:
#Remove Stop words
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
stopwords_list=stopwords.words('english')
stopwords_list.extend(['this','that','where','what','when','why','which','the','they','him','her','includes_bibliographical_references_index'])

#add additional stopwords to be removed

In [ ]:
def clean_text(text):

    ## Remove puncuation
    text = str(text).translate(string.punctuation)


    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\_", " _ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"'\S*@\S*\s?'", '', text)
    text = re.sub(r"'\s+", ' ', text)
    text = re.sub("\'", "", text)
    text=re.sub("-","",text)
    text=re.sub("<","",text)
    text=re.sub(">","",text)
    text=re.sub("</","",text)
    text=re.sub("/>","",text)
    text=re.sub("_","",text)
    text=re.sub(r"\b\d+\b","",text)
    text=re.sub(r"\b\_+\b","",text)
    text = text.lower().split()
    text = [lemmatizer.lemmatize(word) for word in text]
    text = [w for w in text if not w in stopwords_list and len(w) > 3]
    word_dic = Counter(text)
    text = " ".join(word_dic.keys())
    stemmer = PorterStemmer()
    words = word_tokenize(text)
    text = ' '.join([PorterStemmer().stem(word) for word in words])

    return text

In [ ]:
merged['newRisk Sentence']=merged['Risk Sentence'].map(lambda x:clean_text(x))

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
def MorePreprocessing(text):
    text_new = [t for t in text.split() if t not in stop_words]
    text_new = [t for t in text_new if not str.isnumeric(t)]

    return text_new

merged['Risk Sentence new'] = merged['newRisk Sentence'].apply(MorePreprocessing)

In [24]:
#Split into test/train

#Split into test/train
X = merged['Risk Sentence']
y = merged['Label']


print(y)


0        0
1        0
2        0
3        0
4        0
        ..
25532    2
25533    2
25534    2
25535    2
25536    2
Name: Label, Length: 25537, dtype: int64


In [27]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state=42)

y_train = np.asarray(y_train)
y_train

y_test = np.asarray(y_test)

X_train = X_train.to_list()
X_test = X_test.tolist()


print(X_train[:5])


['strive eliminate human error training supervision technology redundant process control', 'information regulatory legislative activity area see privacy data protection data governance information cyber security supervision regulation', 'client demand return cash asset particularly limited notice investment pool liquidity support demand could forced sell investment security held asset pool unfavorable price damaging reputation service provider potentially exposing u claim related management pool', 'continually seek improve assumption model may make modification unintentionally cause le predictive may incorrectly interpret data produced model setting credit policy', 'excess spread le zero certain breach representation warranty covenant agreement relating securitization']


In [9]:
#Bert

len(X_train)
print(len(y_train))

20429


In [10]:
y_train = np.asarray(y_train)
y_train

y_test = np.asarray(y_test)

**Bert**

In [34]:
model_checkpoint = 'bert-base-uncased'

bert_tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
bert_model = TFBertModel.from_pretrained(model_checkpoint)

In [35]:
from transformers import BertTokenizer, BertForSequenceClassification
model_checkpoints_fin = 'yiyanghkust/finbert-tone'

finbert_tokenizer = BertTokenizer.from_pretrained(model_checkpoints_fin)
finbert = BertForSequenceClassification.from_pretrained(model_checkpoints_fin)

In [36]:
#Max

max_length = 100

train_encodings = bert_tokenizer(X_train, truncation=True, padding=True, max_length=max_length, return_tensors='tf')
test_encodings = bert_tokenizer(X_test, truncation=True, padding=True, max_length=max_length, return_tensors='tf')


In [37]:
train_encodings[:1]

{'input_ids': <tf.Tensor: shape=(1, 100), dtype=int32, numpy=
 array([[  101, 29453, 11027,  2529,  7561,  2731, 10429,  2974, 21707,
          2832,  2491,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(1, 100), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [38]:
def create_bert_multiclass_model(checkpoint = model_checkpoint,
                                 num_classes = 4,
                                 hidden_size = 201,
                                 dropout=0.3,
                                 learning_rate=0.00005):
    """
    Build a simple classification model with BERT. Use the Pooler Output for classification purposes.
    """
    bert_model = TFBertModel.from_pretrained(checkpoint)

    ### YOUR CODE HERE
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name ='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}
    bert_out = bert_model(bert_inputs)

    pooler_token = bert_out[1]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_token)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    classification = tf.keras.layers.Dense(num_classes , activation='softmax',name='classification_layer')(hidden)
    #predicted_class = tf.argmax(classification, axis=1)  # Get the index of the class with highest probability
    #predicted_class = tf.cast(predicted_class, dtype=tf.int64)
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs = classification)


    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                 metrics='accuracy')


    ### END YOUR CODE
    return classification_model

In [39]:
pooler_bert_model = create_bert_multiclass_model(checkpoint=model_checkpoint, num_classes=4, hidden_size = 768)

In [40]:
pooler_bert_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask_layer (Inpu  [(None, 100)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 input_ids_layer (InputLaye  [(None, 100)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 100)]                0         []                            
 tLayer)                                                                                    

In [41]:
pooler_bert_model_history = pooler_bert_model.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask],
                                                  y_train,
                                                  validation_data=([test_encodings.input_ids, test_encodings.token_type_ids, test_encodings.attention_mask],
                                                  y_test),
                                                  batch_size=8,
                                                  epochs=1)

2554/2554 [==============================] - 232s 74ms/step - loss: 0.9506 - accuracy: 0.5785 - val_loss: 0.6546 - val_accuracy: 0.7574


In [42]:
pooler_bert_model_history = pooler_bert_model.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask],
                                                  y_train,
                                                  validation_data=([test_encodings.input_ids, test_encodings.token_type_ids, test_encodings.attention_mask],
                                                  y_test),
                                                  batch_size=8,
                                                  epochs=1)

2554/2554 [==============================] - 164s 64ms/step - loss: 0.4865 - accuracy: 0.8258 - val_loss: 0.3779 - val_accuracy: 0.8726


In [54]:
predictions = pooler_bert_model.predict([test_encodings.input_ids, test_encodings.token_type_ids, test_encodings.attention_mask])
predicted_labels = np.argmax(predictions, axis=1)

# Calculate evaluation metrics
classification_rep = classification_report(y_test, predicted_labels)
print(classification_rep)

160/160 [==============================] - 8s 34ms/step
              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1279
           1       0.91      0.82      0.86      1046
           2       0.86      0.86      0.86      1412
           3       0.90      0.92      0.91      1371

    accuracy                           0.87      5108
   macro avg       0.88      0.87      0.87      5108
weighted avg       0.87      0.87      0.87      5108



FINBERT

In [43]:
finbert_train_encodings = finbert_tokenizer(X_train, truncation=True, padding=True, max_length=max_length, return_tensors='tf')
finbert_test_encodings = finbert_tokenizer(X_test, truncation=True, padding=True, max_length=max_length, return_tensors='tf')

In [44]:
finbert_train_encodings[:1]

{'input_ids': <tf.Tensor: shape=(1, 100), dtype=int32, numpy=
 array([[    3,  9246,  3613,  1891,  9221,  2274,  5232,   341, 13578,
           529,   527,     4,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(1, 100), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [49]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

def create_finbert_multiclass_model(checkpoint = model_checkpoints_fin,
                                    num_classes=4,
                                    hidden_size=1024,
                                    dropout=0.3,
                                    learning_rate=0.00005):
    """
    Build a simple classification model with FinBERT. Use the Pooler Output for classification purposes.
    """
    # Load the FinBERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-tone")
    finbert_model = TFBertModel.from_pretrained("yiyanghkust/finbert-tone")

    # Define the inputs
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    # Build the model
    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}
    bert_out = finbert_model(bert_inputs)
    pooler_token = bert_out[1]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_token)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    classification = tf.keras.layers.Dense(num_classes, activation='softmax', name='classification_layer')(hidden)

    # Create the model
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=classification)

    # Compile the model
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                 metrics=['accuracy'])

    return classification_model


In [50]:
finbert_model = create_finbert_multiclass_model(model_checkpoints_fin, num_classes=4, hidden_size = 768)

In [52]:
finbert_model_history = finbert_model.fit([finbert_train_encodings.input_ids, finbert_train_encodings.token_type_ids, finbert_train_encodings.attention_mask],
                                                  y_train,
                                                  validation_data=([finbert_test_encodings.input_ids, finbert_test_encodings.token_type_ids, finbert_test_encodings.attention_mask],
                                                  y_test),
                                                  batch_size=8,
                                                  epochs=1)

2554/2554 [==============================] - 232s 74ms/step - loss: 0.7663 - accuracy: 0.6846 - val_loss: 0.5330 - val_accuracy: 0.8107


In [57]:
# Get predictions from the trained finbert_model
finbert_predictions = finbert_model.predict([finbert_test_encodings.input_ids, finbert_test_encodings.token_type_ids, finbert_test_encodings.attention_mask])
finbert_predicted_labels = np.argmax(finbert_predictions, axis=1)

# Calculate evaluation metrics
finbert_classification_rep = classification_report(y_test, finbert_predicted_labels)
print(finbert_classification_rep)

160/160 [==============================] - 8s 33ms/step
              precision    recall  f1-score   support

           0       0.86      0.80      0.83      1279
           1       0.85      0.79      0.82      1046
           2       0.89      0.70      0.79      1412
           3       0.71      0.94      0.81      1371

    accuracy                           0.81      5108
   macro avg       0.83      0.81      0.81      5108
weighted avg       0.82      0.81      0.81      5108



**CNN **

In [60]:
#CNN
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Flatten
from keras.utils import to_categorical

tokenizer = Tokenizer(num_words=10000)  # Assuming a vocabulary size of 10,000
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_padded = pad_sequences(X_train_seq, maxlen=100, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=100, padding='post')

# Convert labels to one-hot encoded vectors
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

# Build the CNN model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=100, input_length=100))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))  # 4 classes

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
model.fit(X_train_padded, y_train_encoded, validation_data=(X_test_padded, y_test_encoded),
          epochs=10, batch_size=32, verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 conv1d_2 (Conv1D)           (None, 98, 128)           38528     
                                                                 
 global_max_pooling1d_2 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 4)                 260       
                                                                 
Total params: 1047044 (3.99 MB)
Trainable params: 1047044 (3.99 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [61]:
from sklearn.metrics import precision_score, recall_score, f1_score

y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert one-hot encoded test labels back to original labels
y_test_original = np.argmax(y_test_encoded, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test_original, y_pred_classes, average='weighted')
recall = recall_score(y_test_original, y_pred_classes, average='weighted')
f1 = f1_score(y_test_original, y_pred_classes, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

160/160 [==============================] - 0s 1ms/step
Precision: 0.9502848594980123
Recall: 0.9502740798747064
F1-score: 0.9502192330005236
